# Import Library

In [1]:
import os
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
print(tf.config.list_physical_devices('GPU'))
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB4, MobileNetV2

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


#### Set Seed 

In [2]:
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(SEED)

#### Config 

In [3]:
IMAGE_PATH = f'./Crawling/Data/'

batch_size = 16

classes = 17

learning_rate = 5e-5

EPOCHS = 100

# Load Data 

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255.,
)

val_datagen = ImageDataGenerator(
    rescale=1./255.,
)

In [5]:
train_generator = train_datagen.flow_from_directory(
    IMAGE_PATH,
    target_size=[256,256],
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    IMAGE_PATH,
    target_size=[256,256],
    batch_size=batch_size,
    class_mode='categorical'
)

Found 5100 images belonging to 17 classes.
Found 5100 images belonging to 17 classes.


# Build Model 

In [6]:
with tf.device('/device:GPU:0'):
    base_model = MobileNetV2(
        include_top=False,
        input_shape=(256,256,3)
    )
    # base_model = EfficientNetB4(
    #     include_top=False,
    #     input_shape=(256,256,3)
    # )
base_model.trainable = False

In [7]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(17, activation='softmax')

In [8]:
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer,
])

#### Compile Model 

In [9]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 8, 8, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 17)                21777     
Total params: 2,279,761
Trainable params: 21,777
Non-trainable params: 2,257,984
_________________________________________________________________


# Training 

In [ ]:
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator
)

Epoch 1/100
319/319 [==============================] - 16s 51ms/step - loss: 2.8643 - accuracy: 0.1218 - val_loss: 2.5505 - val_accuracy: 0.1978
Epoch 2/100
319/319 [==============================] - 15s 49ms/step - loss: 2.3453 - accuracy: 0.2822 - val_loss: 2.1377 - val_accuracy: 0.3749
Epoch 3/100
319/319 [==============================] - 16s 49ms/step - loss: 1.9961 - accuracy: 0.4310 - val_loss: 1.8461 - val_accuracy: 0.4904
Epoch 4/100
319/319 [==============================] - 16s 49ms/step - loss: 1.7456 - accuracy: 0.5180 - val_loss: 1.6376 - val_accuracy: 0.5594
Epoch 5/100
319/319 [==============================] - 16s 49ms/step - loss: 1.5637 - accuracy: 0.5818 - val_loss: 1.4783 - val_accuracy: 0.6069
Epoch 6/100
319/319 [==============================] - 16s 49ms/step - loss: 1.4259 - accuracy: 0.6222 - val_loss: 1.3584 - val_accuracy: 0.6429
Epoch 7/100
319/319 [==============================] - 16s 49ms/step - loss: 1.3185 - accuracy: 0.6529 - val_loss: 1.2633 - val_ac

In [12]:
r = next(iter(train_generator))

In [13]:
img, label = r[0][0], r[1][0]

In [20]:
model(r[0])

<tf.Tensor: shape=(16, 17), dtype=float32, numpy=
array([[ 2.9310672 , -0.7518295 ,  0.18807179, -1.1059742 ,  1.4337091 ,
         0.62084526,  0.5712175 , -1.9444466 ,  0.17567024, -0.7015872 ,
         0.7865558 , -1.9372294 ,  1.1910641 ,  3.3170536 , -0.94791603,
        -0.63262546, -1.0124608 ],
       [ 3.0522242 , -0.53401685,  0.01308825, -0.5120572 ,  1.5207728 ,
         1.2865975 ,  0.75698715, -1.8376343 , -0.61159384, -1.5248569 ,
         0.52810353, -0.4565504 ,  0.64924103,  3.1790032 , -1.5489216 ,
         1.0847452 , -0.9211453 ],
       [ 1.6581061 ,  0.25070667, -0.74965554, -2.283928  ,  1.0408789 ,
         0.30336374,  0.5115896 , -2.4495153 , -0.18667272, -0.8930874 ,
         2.2171867 , -0.60989845,  0.25996208,  0.8169579 , -0.42487338,
        -0.26345688, -1.1385922 ],
       [ 2.0454292 ,  0.74517614, -1.0220816 , -0.5404272 ,  0.48623416,
         0.7361423 , -0.24974157, -2.5934794 ,  0.6020428 , -1.1695511 ,
         1.3634168 ,  0.68955433,  0.17838